### Project Dashboard developed in Bokeh

run in terminal from root directory<p>

```
bokeh serve --show geo_maping_dashboard.ipynb
```

In [ ]:
import json
from bokeh.io import show, output_file, output_notebook
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Select, RadioButtonGroup, Column, Text, LinearAxis, Grid)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [5]:
# loading chemichal data dataset 
chem_data = pd.read_csv(r'assets/ChemDataForJeffOlson.csv', parse_dates=['VisitDate'])
chem_data['year'] = chem_data.VisitDate.dt.year

seasons = {
    '0':'winter',
    '1':'spring',
    '2':'summer',
    '3':'fall'
}

chem_data['season'] =  (chem_data.VisitDate.dt.month - 1) //3
chem_data['season'] = chem_data.season.apply(lambda x: seasons[str(x)])
chem_data['date'] = chem_data['VisitDate'].apply(lambda x:x.strftime("%Y-%m-01"))

In [6]:
# set treshold for number of measurments
TRESHOLD = 100

def get_frequent(df, TRESHOLD):
    counts = df.groupby(['CharacteristicID'])['Result'].count()
    mask = counts[counts.values <= TRESHOLD].index
    df = df.where(~df.isin(mask))
    return df
              
data = get_frequent(chem_data, TRESHOLD)

In [7]:
df = data.groupby(['LakeID', 'year', 'CharacteristicID'])['Result'].agg(np.mean)
df = df.reset_index()

In [11]:
gdf = gpd.read_file(r'assets/Lake_Inventory_Watersheds.geojson')

In [15]:
def json_data(selectedYear,CharacteristicID):
    
    yr = selectedYear
    df_yr = df[(df['year'] == yr) & (df['CharacteristicID'] == CharacteristicID)]
    
    merged = gdf.merge(df_yr, left_on='LAKEID', right_on='LakeID', how = 'left')
    merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

In [36]:
#Input GeoJSON source that contains features for plotting.
geosource=GeoJSONDataSource(geojson=json_data(1980, "TP"))


#Define a sequential multi-hue color palette.
palette = brewer["YlGnBu"][8]

#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = "#d9d9d9")

#Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 20, height = 550,
                    border_line_color=None,location = (0,0), orientation = "vertical")

#Add hover tool
hover = HoverTool(tooltips = [ ("Lake","@LakeID"),("Result", "@Result")])



#Create figure object.
p = figure(title = "Vermont Lay monitoring programm",
           plot_height = 600,
           plot_width = 500,
           toolbar_location = None,
           tools = [hover],
           x_axis_location=None,
           y_axis_location=None,
          )


p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
p.patches("xs","ys",
          source = geosource,
          fill_color = {"field" :"Result",
                        "transform" : color_mapper},
          line_color = "black",
          line_width = 0.25,
          fill_alpha = 1)



# Define the callback function: update_plot
def update_plot(attr, old, new):
    
    yr = slider.value
    CharacteristicID = parameter_select.value
    
    new_data = json_data(yr, CharacteristicID)
    
    geosource.geojson = new_data
    p.title.text = "Vermont Lay monitoring programm, {}\n Parammeter: {}".format(yr,CharacteristicID)

    
    
# Make a slider object: slider 
slider = Slider(title = "Year",start = 1980, end = 2021, step = 1, value = 1980, width=500)
slider.on_change("value",update_plot)

# Make a drop down object: selector 
LABELS = list(df.CharacteristicID.unique())
parameter_select = Select(value="TP", options=LABELS)
parameter_select.on_change("value", update_plot)
    
    


#Specify layout
p.add_layout(color_bar, "right")


# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider, parameter_select))
curdoc().add_root(layout)

